In [1]:
import allel
from collections import namedtuple
import datetime
import h5py
import ingenos
import itertools
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import collections as mc
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn import model_selection
%matplotlib inline

##### set base directory

In [2]:
base = "/afs/crc.nd.edu/group/BesanskyNGS/data05/comp_karyo"

##### read in data for 2R

In [3]:
v_2R, g_2R = ingenos.import_data(
    "/afs/crc.nd.edu/group/BesanskyNGS2/inversion_genotyping/merged_p2_and_VObs_2R.h5", "2R")

##### read in data for 2L

In [4]:
path_2L = "/afs/crc.nd.edu/group/BesanskyNGS2/inversion_genotyping/merged_p2_and_VObs_2L.h5"
chrom_2L = "2L"

callset_2L = h5py.File(path_2L, mode='r')[chrom_2L]

v_2L = allel.VariantChunkedTable(callset_2L['variants'], index='POS',
                                names=['POS','REF','ALT','DP','MQ','QD','numalt'])

g_2L = allel.GenotypeChunkedArray(callset_2L['calldata']['GT'])

##### read in metadata

In [5]:
md_2L = pd.read_csv(base + "/metadata/all_samples_2L_metadata_080318.csv", sep="\t")
md_2R = pd.read_csv(base + "/metadata/all_samples_2R_metadata_080318.csv", sep="\t")

##### read in the top SNPs.

In [6]:
a_top = pd.read_csv(base + "/data/results/2La/comp/predictive_SNPs_train_set_0995_110918.tsv",
               sep = "\t", header=None)

j_top = pd.read_csv(base + "/data/results/2Rj/comp/predictive_SNPs_train_set_08_110918.tsv",
               sep = "\t", header=None)

b_top = pd.read_csv(base + "/data/results/2Rb/comp/predictive_SNPs_train_set_08_110918.tsv",
               sep = "\t", header=None)

c_col_top = pd.read_csv(base + "/data/results/2Rc/comp/col_predictive_SNPs_train_set_08_031919.tsv", 
                      sep="\t", header=None)

c_gam_top = pd.read_csv(base + "/data/results/2Rc/comp/gam_ss_predictive_SNPs_train_set_08_031919.tsv", 
                      sep="\t", header=None)

u_top = pd.read_csv(base + "/data/results/2Ru/comp/predictive_SNPs_train_set_08_110918.tsv",
               sep = "\t", header=None)

d_top = pd.read_csv(base + "/data/results/2Rd/comp/predictive_SNPs_train_set_08_052619.tsv",
               sep = "\t", header=None)

##### read in the training-validation splits to keep only the validation set

In [7]:
splits = np.load(base + "/metadata/comp_karyo_splits/splits.npy", allow_pickle=True).flat[0]

splits_d = np.load(base + "/metadata/comp_karyo_splits/2Rdj_splits.npy",
                allow_pickle=True).flat[0]

splits["2Rd"] = splits_d["2Rd"]

##### mask low-quality genotypes.

In [8]:
merged_2R = h5py.File(
    "/afs/crc.nd.edu/group/BesanskyNGS2/inversion_genotyping/merged_p2_and_VObs_2R.h5", 
    mode="r")

gq_2R = merged_2R["2R"]['calldata']['GQ'][:]

g_2R.mask = gq_2R < 20

In [9]:
gq_2L = callset_2L['calldata']['GQ'][:]

g_2L.mask = gq_2L < 20

##### subset the data and the metadata

In [10]:
g_2La = g_2L.subset(sel1 = md_2L["ox_code"].isin(splits["2La"]["test"]).values)
g_2Rj = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rj"]["test"]).values)
g_2Rb = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rb"]["test"]).values)
g_2Rc = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rc"]["test"]).values)
g_2Rd = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rd"]["test"]).values)
g_2Ru = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Ru"]["test"]).values)

md_2La = md_2L.loc[md_2L["ox_code"].isin(splits["2La"]["test"]),:]
md_2Rb = md_2R.loc[md_2R["ox_code"].isin(splits["2Rb"]["test"]),:]
md_2Rc = md_2R.loc[md_2R["ox_code"].isin(splits["2Rc"]["test"]),:]
md_2Rd = md_2R.loc[md_2R["ox_code"].isin(splits["2Rd"]["test"]),:]
md_2Rj = md_2R.loc[md_2R["ox_code"].isin(splits["2Rj"]["test"]),:]
md_2Ru = md_2R.loc[md_2R["ox_code"].isin(splits["2Ru"]["test"]),:]

##### then, subset by species for 2Rc. we also drop the Bamako specimens (Bamako is a chromosomal form defined by 2Rjcu == 2, with 2Rb polymorphic; in our data set, all gambiae specimens with 2Rj == 2 are Bamako, so we can use that as a shortcut); Bamako appears to be on an independent evolutionary trajectory

In [11]:
g_2Rc = g_2Rc.subset(sel1 = md_2Rc["2Rj"] != "2.0")
md_2Rc = md_2Rc.loc[md_2Rc["2Rj"] != "2.0",:]

col_bool = (md_2Rc["species"] == "An. coluzzii").values
gam_bool = (md_2Rc["species"] == "An. gambiae").values

g_2Rc_col = g_2Rc.subset(sel1 = col_bool)
g_2Rc_gam = g_2Rc.subset(sel1 = gam_bool)

col_md = md_2Rc.loc[col_bool,:]
gam_md = md_2Rc.loc[gam_bool,:]

In [12]:
Inversion = namedtuple('Inversion',['SNPs','metadata','genotypes','inv_title'])

In [13]:
inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = md_2La, genotypes = g_2La,
                             inv_title = "new_PCA_2La"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = md_2Rb, genotypes = g_2Rb,
                            inv_title = "new_PCA_2Rb"),
            "2Rc_col" : Inversion(SNPs = c_col_top.values, metadata = col_md, 
                                 genotypes = g_2Rc_col,
                             inv_title = "new_PCA_2Rc"),
           "2Rc_gam" : Inversion(SNPs = c_gam_top.values, metadata = gam_md, 
                                genotypes = g_2Rc_gam,
                            inv_title = "new_PCA_2Rc")
           "2Rd" : Inversion(SNPs = d_top.values, metadata = md_2Rd, genotypes = g_2Rd,
                            inv_title = "new_PCA_2Rd"),
           "2Rj" : Inversion(SNPs = j_top.values, metadata = md_2Rj, genotypes = g_2Rj,
                            inv_title = "new_PCA_2Rj"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = md_2Ru, genotypes = g_2Ru,
                            inv_title = "new_PCA_2Ru")}

SyntaxError: invalid syntax (<ipython-input-13-1119e6cb0aa1>, line 11)

In [ ]:
for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    
    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    
    mismatches = np.sum(md[new_col_name] != md[col_name].map(float).map(int))
    
    print(inversion, " # mismatches: ", mismatches,"\n")
    print(av_gts)
    print(total_sites,"\n")

for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    called_name = inversion + "_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"

    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    md[called_name] = total_sites
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[called_name]
    
    print(inversion,"\n")
    print(av_gts)
    print(total_sites,"\n")

##### repeat for all specimens to karyotype them all

In [ ]:
inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = md_2L, genotypes = g_2L,
                             inv_title = "new_PCA_2La"),
            "2Rj" : Inversion(SNPs = j_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rj"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rb"),
            "2Rc_col" : Inversion(SNPs = c_col_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rc"),
           "2Rc_gam" : Inversion(SNPs = c_gam_top.values, metadata = md_2R, genotypes = g_2R, 
                            inv_title = "new_PCA_2Rc"),
           "2Rd" : Inversion(SNPs = d_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rd"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Ru")}

In [ ]:
for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    called_name = inversion + "_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"

    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    md[called_name] = total_sites
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[called_name]
    
    print(inversion,"\n")
    print(av_gts)
    print(total_sites,"\n")

##### save these two metadata files to the desired locations, as they now contain results